# Pipeline Beispiel

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

## Erfassung der gewünschten Wertpapiere

### Der Q500US und Q1500US

Diese Gruppen handelbare Aktien werden als "Universen" bezeichnet, da all Deine Trades diese Aktien als ihr "Universum" der verfügbaren Aktien benutzen werden; sie werden mit nichts außerhalb dieser Gruppen handeln.

In [16]:
from quantopian.pipeline.filters import Q1500US

Es gibt zwei große Vorteile von Q500US und Q1500US. Erstens senken sie das Risiko, dass ein Auftrag nicht ausgeführt wird, immens. Zweitens erlauben sie bedeutendere Vergleiche zwischen Strategien, da sie nun als als Standarduniversen für Algorithmen verwendet werden.

In [17]:
universe = Q1500US()

## Weitere Filterung des Universums mit Klassifikatoren

Wir betrachten nur Posten aus dem Energiesektor: https://www.quantopian.com/help/fundamentals#industry-sector

In [25]:
from quantopian.pipeline.data import morningstar

In [29]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

Alternative:

In [30]:
#from quantopian.pipeline.classifiers.morningstar import Sector
#morningstar_sector = Sector()

In [36]:
energy_sector = sector.eq(309)

## Filter maskieren

Masken können auch auf Methoden angewendet werden, die Filter zurückgeben, wie top, bottom und percentile_between.

Masken sind am nützlichsten, wenn wir einen Filter in der ersten Schritten einer zusammengesetzten Berechnung anwenden wollen. Nehmen wir beispielsweise an, dass wir die 50 Wertpapiere mit den höchsten Öffnungspreisen finden wollen, die auch in den top 10 % des Dollarvolumens liegen.

Nehmen wir an, dass wir das 90.-100. Perzentil dieser Wertpapiere anhand des Schlusspreises haben wollen. Dies können wir wie folgt bekommen:

In [38]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [37]:
# Faktor Dollarvolumen
dollar_volume = AverageDollarVolume(window_length=30)

# Filter nach hohem Dollarvolumen
high_dollar_volume = dollar_volume.percentile_between(90,100)

# Filter nach höchstem Öffnungspreis (Wertpapiere mit hohem Dollarvolumen)
top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

# Filter nach oberstem Perzentil des Schlusspreises (hohes Dollarvolumen, top 50 Öffnungspreis)
high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)

## Filter und Faktoren anwenden

Wenden wir unsere eigenen Filter auf ein paar der obigen Beispiele an. Dazu wählen wir Wertpapiere mit folgenden Eigenschaften:

* Aktien im Q1500US
* Aktien im Energiesektor
* relativ hoch gehandelte Aktien im Markt (anhand des gehandelten Dollarvolumens, muss in den Top 5% der gehandelten Aktien sein)

Dann berechnen wir den prozentualen Unterschied, wie wir es zuvor schon getan haben. Diesen prozentualen Unterschied verwenden wir, um eine unkomplizierte Strategie zu entwickeln, die alles mit negativem prozentualen Unterschied kürzt  (Unterschied zwischen 10-Tages- und 30-Tages-Durchschnitt).

In [73]:
def make_pipeline():
    
    # Grunduniversum filtern
    base_universe = Q1500US()
    
    # Sektorklassifikator als Filter
    energy_sector = sector.eq(309)
    
    # Maskierung Grundenergieaktien
    base_energy = base_universe & energy_sector
    
    # Faktor Dollarvolumen
    dollar_volume = AverageDollarVolume(window_length=30)

    # Obere Hälfte des Dollarvolumenfilters
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # Finale Filtermaske
    top_half_base_energy = base_energy & high_dollar_volume
    
    # 10-Tages-Schlusspreisdurchschnitt
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=top_half_base_energy)

    # 30-Tages-Schlusspreisdurchschnitt
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=top_half_base_energy)

    # Faktor prozentualer Unterschied
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # Erstellen eines Filters, um die zu kürzenden Wertpapiere auszuwählen
    shorts = percent_difference < 0
    
    # Erstellen eines Filters, um die zu verlängernden Wertpapiere auszuwählen
    longs = percent_difference > 0
    
    # Filter für die Wertpapiere, die wir handeln wollen
    securities_to_trade = (shorts | longs)
    
    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts,
            'percent_diff':percent_difference
        },
        screen=securities_to_trade
    )

In [74]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

longs  percent_diff shorts
2015-05-05 00:00:00+00:00 Equity(216 [HES])     True      0.036528  False
                          Equity(448 [APA])     True      0.035926  False
                          Equity(455 [APC])     True      0.049153  False
                          Equity(858 [BHI])     True      0.033807  False
                          Equity(1746 [COG])    True      0.058078  False
                          Equity(2368 [DVN])    True      0.046264  False
                          Equity(2564 [EOG])    True      0.032102  False
                          Equity(2621 [ESV])    True      0.060197  False
                          Equity(3443 [HAL])    True      0.049257  False
                          Equity(3647 [HP])     True      0.040991  False
                          Equity(5035 [MRO])    True      0.061598  False
                          Equity(5213 [NBL])    True      0.010443  False
                          Equity(5214 [NBR])    True      0.064133  False
                          Equity(5249 [NE])     True      0.037559  False
                          Equity(5729 [OXY])    True      0.029776  False
                          Equity(6928 [SLB])    True      0.046555  False
                          Equity(7244 [SWN])    True      0.070788  False
                          Equity(7612 [ANDV])   True      0.005997  False
                          Equity(7990 [VLO])   False     -0.017145   True
                          Equity(8214 [WMB])    True      0.018876  False
                          Equity(8347 [XOM])    True      0.017343  False
                          Equity(8461 [CHK])    True      0.014265  False
                          Equity(9038 [RIG])    True      0.048180  False
                          Equity(13176 [CAM])   True      0.082110  False
                          Equity(17436 [PXD])   True      0.010248  False
                          Equity(19249 [RRC])   True      0.087062  False
                          Equity(19336 [WFT])   True      0.049141  False
                          Equity(22784 [FTI])   True      0.054529  False
                          Equity(23112 [CVX])   True      0.018972  False
                          Equity(23998 [COP])   True      0.023902  False
                          Equity(24809 [NOV])   True      0.024940  False
                          Equity(25707 [WLL])   True      0.048205  False
                          Equity(33856 [CLR])   True      0.064304  False
                          Equity(34440 [CXO])   True      0.042184  False
                          Equity(39797 [OAS])   True      0.042388  False
                          Equity(40852 [KMI])   True      0.023016  False
                          Equity(41636 [MPC])   True      0.011952  False
                          Equity(42788 [PSX])   True      0.020911  False

In [75]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 38 entries, (2015-05-05 00:00:00+00:00, Equity(216 [HES])) to (2015-05-05 00:00:00+00:00, Equity(42788 [PSX]))
Data columns (total 3 columns):
longs           38 non-null bool
percent_diff    38 non-null float64
shorts          38 non-null bool
dtypes: bool(2), float64(1)
memory usage: 684.0+ bytes


# Ausführung dieser Strategie in der IDE

In [ ]:
from quantopian.algorithm import attach_pipeline,pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume,SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar

def initialize(context):
    
    schedule_function(my_rebalance,date_rules.week_start(),time_rules.market_open(hours=1))
    
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe,'my_pipeline')
    
def my_rebalance(context,data):
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security,0)
            
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security,context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security,context.short_weight)




def my_compute_weights(context):
    
    if len(context.longs)==0:
        long_weight = 0
    else:
        long_weight = 0.5 / len(context.longs)
  
    if len(context.shorts)==0:
        short_weight = 0
    else:
        short_weight = 0.5 / len(context.shorts)
    
    return (long_weight,short_weight)






def before_trading_start(context,data):
    context.output = pipeline_output('my_pipeline')
    
    # LONG
    context.longs = context.output[context.output['longs']].index.tolist()
    
    # SHORT
    context.shorts = context.output[context.output['shorts']].index.tolist()


    context.long_weight,context.short_weight = my_compute_weights(context)



def make_pipeline():
    
    # Universum Q1500US
    base_universe = Q1500US()
    
    # Energiesektor
    sector = morningstar.asset_classification.morningstar_sector_code.latest
    energy_sector = sector.eq(309)
    
    # Maske von Q1500US und Energie erstellen
    base_energy = base_universe & energy_sector
    
    # Dollarvolumen (30 Tage) Informationen erfassen
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Erfasse die oberen 5% anhand des durchschnittlichen Dollarvolumens
    high_dollar_volume = dollar_volume.percentile_between(95,100)
     
    # Filter kombinieren
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10-Tages-Schlusspreisdurchschnitt
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=top_five_base_energy)
    
    # 30-Tages-Schlusspreisdurchschnitt
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=top_five_base_energy)
    
    # prozentualer Unterschied
    percent_difference = (mean_10-mean_30)/mean_30
    
    # Liste der Short-Verkäufe
    shorts = percent_difference < 0
    
    # Liste der Long-Verkäufe
    longs = percent_difference > 0
    
    # Finale Maske/Filter für alles in shorts oder longs
    securities_to_trade = (shorts | longs)
    
    # Pipeline zurückgeben
    return Pipeline(columns={
        'longs':longs,
        'shorts':shorts,
        'perc_diff':percent_difference
    },screen=securities_to_trade)